# 3.2 Data gathering

In [2]:
import subprocess
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json

In [3]:
pd.set_option('display.max_rows',500)

# Github csv data

In [3]:
git_pull = subprocess.Popen( "/usr/bin/git pull" , 
                     cwd = os.path.dirname( '../data/raw/COVID-19/' ), 
                     shell = True, 
                     stdout = subprocess.PIPE, 
                     stderr = subprocess.PIPE )
(out, error) = git_pull.communicate()


print("Error : " + str(error)) 
print("out : " + str(out))

Error : b'From https://github.com/CSSEGISandData/COVID-19\n   4070863d..d07a81e2  master     -> origin/master\n   d6349303..ce6d6d9d  web-data   -> origin/web-data\n'
out : b'Updating 4070863d..d07a81e2\nFast-forward\n README.md                                          |    7 +\n csse_covid_19_data/README.md                       |    9 +-\n csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv   |    2 +-\n .../csse_covid_19_daily_reports/08-29-2020.csv     | 3951 ++++++++++++\n .../csse_covid_19_daily_reports/08-30-2020.csv     | 3951 ++++++++++++\n .../csse_covid_19_daily_reports/08-31-2020.csv     | 3955 ++++++++++++\n .../csse_covid_19_daily_reports/09-01-2020.csv     | 3955 ++++++++++++\n .../csse_covid_19_daily_reports/09-02-2020.csv     | 3955 ++++++++++++\n .../csse_covid_19_daily_reports_us/08-29-2020.csv  |   59 +\n .../csse_covid_19_daily_reports_us/08-30-2020.csv  |   59 +\n .../csse_covid_19_daily_reports_us/08-31-2020.csv  |   59 +\n .../csse_covid_19_daily_reports_us/09-01-2

In [4]:
data_path = '/home/shreeka/Enterprise Data Science/covid-19 project/data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
pd.read_csv(data_path)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,37599,37856,37894,37953,37999,38054,38070,38113,38129,38140
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,7812,7967,8119,8275,8427,8605,8759,8927,9083,9195
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,39847,40258,40667,41068,41460,41858,42228,42619,43016,43403
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,1024,1024,1045,1045,1045,1060,1060,1098,1098,1124
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,2015,2044,2068,2134,2171,2222,2283,2332,2415,2471
5,NaN,Antigua and Barbuda,17.060800,-61.796400,0,0,0,0,0,0,...,94,94,94,94,94,94,94,94,94,94
6,NaN,Argentina,-38.416100,-63.616700,0,0,0,0,0,0,...,312659,320884,329043,336802,342154,350867,359638,370188,380292,392009
7,NaN,Armenia,40.069100,45.038200,0,0,0,0,0,0,...,42056,42319,42477,42616,42792,42825,42936,43067,43270,43451
8,Australian Capital Territory,Australia,-35.473500,149.012400,0,0,0,0,0,0,...,113,113,113,113,113,113,113,113,113,113
9,New South Wales,Australia,-33.868800,151.209300,0,0,0,0,3,4,...,3971,3972,3981,3985,3988,3991,3997,4006,4019,4033


# 3.3 Web scraping

In [5]:
page = requests.get("https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html")

In [6]:
soup = BeautifulSoup(page.content, 'html.parser')

In [7]:
html_table=soup.find('table')

In [10]:
all_rows=html_table.find_all('tr')

In [16]:
final_data_list=[]
for pos,rows in enumerate(all_rows):
   col_list=[each_col.get_text(strip=True) for each_col in rows.find_all('td')]
   final_data_list.append(col_list) 

In [20]:
pd_daily_status = pd.DataFrame(final_data_list).dropna().rename(columns={0:'state',
                                                       1:'cases',
                                                       2:'changes',
                                                       3:'cases_per_100k',
                                                       4:'fatal',
                                                       5:'comment'})

In [21]:
pd_daily_status.head()

,state,cases,changes,cases_per_100k,fatal,comment
2,Baden-Württem­berg,42.119,+177,1.543,"13,9",1.866
3,Bayern,57.485,+266,1.839,"14,1",2.640
4,Berlin,11.266,+77,435,"11,6",226
5,Branden­burg,3.871,+1,50,"2,0",169
6,Bremen,2.002,+9,90,"13,2",56


# 3.4 REST API

In [3]:
api_page = requests.get("https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronaf%C3%A4lle_in_den_Bundesl%C3%A4ndern/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json")

ConnectionError: HTTPSConnectionPool(host='services7.arcgis.com', port=443): Max retries exceeded with url: /mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronaf%C3%A4lle_in_den_Bundesl%C3%A4ndern/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fd4bffd18b0>: Failed to establish a new connection: [Errno -2] Name or service not known'))

In [8]:
json_object = json.loads(api_page.content)

In [10]:
json_object.keys()

dict_keys(['objectIdFieldName', 'uniqueIdField', 'globalIdFieldName', 'geometryProperties', 'serverGens', 'geometryType', 'spatialReference', 'fields', 'features'])

In [15]:
full_list = []
for count,ele in enumerate(json_object['features'][:]):
    full_list.append(ele['attributes'])

In [16]:
pd_fullList = pd.DataFrame(full_list)

In [18]:
pd_fullList.head()

,OBJECTID_1,LAN_ew_AGS,LAN_ew_GEN,LAN_ew_BEZ,LAN_ew_EWZ,OBJECTID,Fallzahl,Aktualisierung,AGS_TXT,GlobalID,faelle_100000_EW,Shape__Area,Shape__Length,Death
0,1,01,Schleswig-Holstein,Land,2896712,15,4064,1598997600000,01,fc5ba936-c95c-432c-8a33-9eb2f30b660f,140.296999,4.573731e+10,2.881496e+06,160
1,2,02,Hamburg,Freie und Hansestadt,1841179,6,6313,1598997600000,02,0f3e860c-5181-4d3f-a421-1d51f50315ea,342.878123,2.089396e+09,4.188002e+05,267
2,3,03,Niedersachsen,Land,7982448,9,16888,1598997600000,03,3fd77024-c29b-4843-9be8-682ad48e60c9,211.564172,1.299836e+11,4.008988e+06,662
3,4,04,Bremen,Freie Hansestadt,682986,5,2015,1598997600000,04,4132268b-54de-4327-ac1e-760e915112f1,295.028009,1.119157e+09,3.357177e+05,57
4,5,05,Nordrhein-Westfalen,Land,17932651,10,59263,1598997600000,05,561d658f-3ee5-46e3-bc95-3528c6558ab9,330.475399,8.782936e+10,2.648673e+06,1817


# 3.5 REST service for US data

In [2]:
headers = {
# Request headers
'Cache-Control': 'no-cache',
'Subscription-Key': '4afc83d956644476989607f36078a3c8',
}

response_us = requests.get('https://api.smartable.ai/coronavirus/stats/US',headers=headers)
print (response_us)


<Response [200]>


In [6]:
us_dict = json.loads(response_us.content)
with open('../data/raw/SMARTABLE/US_data.json', 'w') as outfile:
    json.dump(us_dict, outfile,indent=2)